In [19]:
import sys
import pandas as pd
import numpy as np

import re
from joblib import dump, load
import matplotlib.pyplot as plt
pd.options.display.max_colwidth=200
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import PolynomialFeatures

import scipy.stats as stats
import seaborn as sns; sns.set()  # for plot styling
from mpl_toolkits.mplot3d import Axes3D # for 3D plots
from sklearn.model_selection import KFold, GridSearchCV, train_test_split
from sklearn.preprocessing import OneHotEncoder
import seaborn as sns
from sklearn.metrics import mean_squared_error as mse, r2_score, mean_absolute_error as mae
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder

# Metricas
from sklearn.metrics import confusion_matrix, classification_report, precision_score, recall_score, f1_score, accuracy_score
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.neighbors import KNeighborsClassifier

# q-q plots
import scipy.stats as stats

# !!!! + AGRUPACION
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering



from scipy.spatial.distance import cdist
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.preprocessing import MinMaxScaler
import matplotlib.cm as cm



# Carga de datos

In [20]:
df = pd.read_csv('data/base_2_cotizaciones.txt', sep = '\t')


In [21]:
df_cot = df.copy()

# Entendimiento específico

In [22]:
df_cot.head(5)

,cotizacion,estado_cotizacion,id,fecha_creacion,fecha_modificacion,categoria_macro,categoria,producto,cantidad,precio,valor
0,1381861.0,Cotización Ganada,91587,1972-08-05 12:25:47.700,1972-08-05 13:37:35,categoria_macro_4,categoria_9,producto_524,1.0,34.957429,34.957429
1,1381861.0,Cotización Ganada,91587,1972-08-05 12:25:47.700,1972-08-05 13:37:35,categoria_macro_4,categoria_11,producto_124,1.0,2.262000,2.262000
2,1381861.0,Cotización Ganada,91587,1972-08-05 12:25:47.700,1972-08-05 13:37:35,categoria_macro_4,categoria_10,producto_5085,1.0,40.600000,40.600000
3,1381861.0,Cotización Ganada,91587,1972-08-05 12:25:47.700,1972-08-05 13:37:35,categoria_macro_4,categoria_11,producto_188,1.0,7.863143,7.863143
4,1381864.0,Cotización Ganada,93623,1972-08-05 14:36:57.273,1972-08-05 15:06:26,categoria_macro_4,categoria_10,producto_391,1.0,63.932571,63.932571


In [23]:
productos = df_cot["producto"].unique().size
print(f"Cantidad Productos: {productos}")

Cantidad Productos: 2735


In [24]:
estados = df_cot["estado_cotizacion"].unique()
print(f"Estados: {estados}")

Estados: ['Cotización Ganada' 'Cotización Expirada' 'Cotización Perdida' 'Abierta']


# DF de precios 

Para los productos con varios precios asignados se calculó el pormedio para tener la tabla de preciosos son con productos únicos.

In [25]:

df_precios = df_cot[['producto', 'precio']].drop_duplicates()
df_precios = df_precios.groupby('producto', as_index=False)['precio'].mean()


print(df_precios)


           producto     precio
0        producto_1   8.749715
1       producto_10  11.696667
2      producto_100   2.038286
3     producto_1002   9.106000
4     producto_1003   0.472286
...             ...        ...
2730   producto_992  39.611238
2731   producto_993  56.442285
2732   producto_995  10.348857
2733   producto_996  21.244571
2734   producto_998  27.210286

[2735 rows x 2 columns]


# DF cotizaciones fallidas

In [26]:

df_filtrado = df_cot[df_cot['estado_cotizacion'].isin(['Cotización Perdida', 'Cotización Expirada'])]
